# TensorFlow practice 1

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
import tensorflow as tf
import datetime
import pandas as pd
import numpy as np

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
    return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

In [4]:
#mnist.load_data()

In [5]:
# this why input_shap=(28, 28)
x_train.shape

(60000, 28, 28)

In [6]:
x_train[0,0,]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [7]:
y_train.shape

(60000,)

In [8]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [9]:
# find out the unique values, this is why output dense is 10
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [10]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=70, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Epoch 1/70
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2201 - accuracy: 0.9351 - val_loss: 0.1171 - val_accuracy: 0.9635
Epoch 2/70
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0980 - accuracy: 0.9705 - val_loss: 0.0744 - val_accuracy: 0.9770
Epoch 3/70
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0685 - accuracy: 0.9786 - val_loss: 0.0716 - val_accuracy: 0.9792
Epoch 4/70
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0513 - accuracy: 0.9839 - val_loss: 0.0724 - val_accuracy: 0.9772
Epoch 5/70
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0451 - accuracy: 0.9855 - val_loss: 0.0673 - val_accuracy: 0.9794
Epoch 6/70
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0376 - accuracy: 0.9879 - val_loss: 0.0657 - val_accuracy: 0.9823
Epoch 7/70
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0310 - accuracy: 0.9891 - val_loss: 0.0638 - val_accuracy:

1875/1875 [==============================] - 7s 4ms/step - loss: 0.0080 - accuracy: 0.9977 - val_loss: 0.1654 - val_accuracy: 0.9827
Epoch 58/70
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0066 - accuracy: 0.9981 - val_loss: 0.1710 - val_accuracy: 0.9828
Epoch 59/70
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0097 - accuracy: 0.9975 - val_loss: 0.1663 - val_accuracy: 0.9847
Epoch 60/70
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0079 - accuracy: 0.9979 - val_loss: 0.1810 - val_accuracy: 0.9834
Epoch 61/70
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0071 - accuracy: 0.9981 - val_loss: 0.1662 - val_accuracy: 0.9845
Epoch 62/70
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0078 - accuracy: 0.9978 - val_loss: 0.1806 - val_accuracy: 0.9848
Epoch 63/70
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0088 - accuracy: 0.9978 - val_loss: 0.1764 - val_accuracy: 0.98

In [11]:
%tensorboard --logdir logs/fit

## Using TensorBoard with other methods

In [12]:
#Use the same dataset as above, but convert it to tf.data.Dataset to take advantage of batching capabilities:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

In [13]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [14]:
#Create stateful metrics that can be used to accumulate values during training and logged at any point:

# Define our metrics
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [15]:
#Define the training and test functions:
def train_step(model, optimizer, x_train, y_train):
    with tf.GradientTape() as tape:
        predictions = model(x_train, training=True)
        loss = loss_object(y_train, predictions)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        train_loss(loss)
        train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
    predictions = model(x_test)
    loss = loss_object(y_test, predictions)

    test_loss(loss)
    test_accuracy(y_test, predictions)

In [16]:
#Set up summary writers to write the summaries to disk in a different logs directory:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [17]:
model = create_model() # reset our model

EPOCHS = 8

for epoch in range(EPOCHS):
    for (x_train, y_train) in train_dataset:
        train_step(model, optimizer, x_train, y_train)
        with train_summary_writer.as_default():
            tf.summary.scalar('loss', train_loss.result(), step=epoch)
            tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

    for (x_test, y_test) in test_dataset:
        test_step(model, x_test, y_test)
        with test_summary_writer.as_default():
            tf.summary.scalar('loss', test_loss.result(), step=epoch)
            tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1,
                         train_loss.result(), 
                         train_accuracy.result()*100,
                         test_loss.result(), 
                         test_accuracy.result()*100))
    # Reset metrics every epoch
    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()


Epoch 1, Loss: 0.24611052870750427, Accuracy: 92.70166778564453, Test Loss: 0.1238837018609047, Test Accuracy: 96.21000671386719
Epoch 2, Loss: 0.10494938492774963, Accuracy: 96.86000061035156, Test Loss: 0.08949847519397736, Test Accuracy: 97.05999755859375
Epoch 3, Loss: 0.0724712684750557, Accuracy: 97.788330078125, Test Loss: 0.07162424921989441, Test Accuracy: 97.86000061035156
Epoch 4, Loss: 0.0551287904381752, Accuracy: 98.2550048828125, Test Loss: 0.06464751809835434, Test Accuracy: 98.0
Epoch 5, Loss: 0.043045539408922195, Accuracy: 98.6199951171875, Test Loss: 0.07003224641084671, Test Accuracy: 97.69999694824219
Epoch 6, Loss: 0.03422348573803902, Accuracy: 98.9433364868164, Test Loss: 0.062081124633550644, Test Accuracy: 98.1500015258789
Epoch 7, Loss: 0.030936535447835922, Accuracy: 99.04000091552734, Test Loss: 0.056482456624507904, Test Accuracy: 98.33999633789062
Epoch 8, Loss: 0.02610090747475624, Accuracy: 99.1550064086914, Test Loss: 0.06350827217102051, Test Accurac

In [18]:
%tensorboard --logdir logs/gradient_tape

## Save and reload trained model

In [19]:
# Save the model
model.save("adv_model_trained.h5")

In [20]:
# Load the model
from tensorflow.keras.models import load_model
adv_model = load_model("adv_model_trained.h5")

In [21]:
#reloaded model must compile before use
adv_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## evaluate reloaded model

In [22]:
model_loss, model_accuracy = adv_model.evaluate(
    x_test, y_test, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

1/1 - 0s - loss: 3.2503e-04 - accuracy: 1.0000
Normal Neural Network - Loss: 0.0003250319859944284, Accuracy: 1.0
